# Telecommunication Network Optimization

In [1]:
#Import packages and data

import openpyxl
import pulp
from pulp import *

path = "InputDataTelecomSmallInstance.xlsx"

wb = openpyxl.load_workbook(path)
sheets = wb.sheetnames

In [2]:
#Define the set of parameters

C = wb[sheets[0]]
number_of_customers = C['A1'].value
Customers = range(1,number_of_customers+1)

M = wb[sheets[1]]
end_offices = M['A1'].value
End_offices = range(1,end_offices+1)

A = wb[sheets[2]]
alpha = A['A1'].value

N = wb[sheets[3]]
number_of_hub = N['A1'].value
Hubs = range(1,number_of_hub+1)

CustToTargetAllocCost = wb[sheets[4]]
h_index = {}
for customer in Customers:
    for office in End_offices:
        h_index[(customer,office)] = CustToTargetAllocCost.cell(row=customer,column=office).value

TargetToSteinerAllocCost = wb[sheets[5]]
c_index = {}
for office in End_offices:
    for hub in Hubs:
        c_index[(office,hub)] = TargetToSteinerAllocCost.cell(row=office,column=hub).value

SteinerToSteinerConnctCost = wb[sheets[6]]
g_index = {}
for hub1 in Hubs:
    for hub2 in Hubs:
        g_index[(hub1,hub2)] = SteinerToSteinerConnctCost.cell(row=hub1,column=hub2).value

SteinerFixedCost = wb[sheets[7]]
fixed_coast = {}
for hub in Hubs:
    fixed_coast[hub] = SteinerFixedCost.cell(row=hub,column=1).value

TargetCapicity = wb[sheets[8]]
u_index = {}
for office in End_offices:
    u_index[office] = TargetCapicity.cell(row=office,column=1).value

SteinerCapacity = wb[sheets[9]]
v_index = {}
for hub in Hubs:
    v_index[hub] = SteinerCapacity.cell(row=hub,column=1).value

In [31]:
print(v_index)

{1: 6, 2: 5, 3: 7, 4: 6, 5: 5, 6: 6}


In [14]:
#Create decision variables
X_var= LpVariable.dicts('X',(Customers,End_offices),0,cat='Binary')

Y_var = LpVariable.dicts('Y',(End_offices,Hubs),0,cat='Binary')

Z_var = LpVariable.dicts('Z',(Hubs,Hubs),0,cat='Binary')

L_var = LpVariable.dicts('L',Hubs,0,cat='Binary')

W_var = LpVariable.dicts('W',(Customers,End_offices,Hubs),0,cat='Binary')

#Create the problem
problem = LpProblem("problem", LpMinimize)

#add the objective function
problem += lpSum(h_index[key]*X_var[key[0]][key[1]] for key in h_index)+lpSum(c_index[key]*X_var[key[0]][key[1]] for key in c_index)+lpSum(fixed_coast[hub]*L_var[hub] for hub in Hubs)+lpSum(g_index[key]*Z_var[key[0]][key[1]] for key in g_index)

In [15]:
def part_list(l):
    sequence = []
    i, imax = 0, 2**len(l)-1
    while i <= imax:
        s = []
        j, jmax = 0, len(l)-1
        while j <= jmax:
            if (i>>j)&1 == 1:
                s.append(l[j])
            j += 1
        sequence.append(s)
        i += 1
    return sequence

def substract(A,B):
    #A - B
    l = []
    assert len(A)>=len(B)
    for i in A:
        if i not in B:
            l.append(i)
    return l

In [47]:
def dot(l1,l2):
    d=[]
    for i in l1:
        for j in l2:
            d.append((i,j))
    return d

In [49]:
#Add constraints

#Constraint 1
for customer in Customers:
    problem += lpSum(X_var[customer][office] for office in End_offices) <= 1
    
#Constraint 2
for office in End_offices:
    problem += lpSum(Y_var[office][hub] for hub in Hubs) == 1
    
#Constraint 3
for hub in Hubs:
    for office in End_offices:
        problem += Y_var[office][hub] <= L_var[hub]
        
#Constraint 4
for hub1 in Hubs:
    problem += lpSum(Z_var[hub2][hub1] for hub2 in Hubs) + lpSum(Z_var[hub1][hub2] for hub2 in Hubs) == 2*L_var[hub1]

#Constraint 5
Hub_part = part_list(Hubs)
for part in Hub_part:
    if (len(part)>=3) and (len(part)<=lpSum(L_var[hub] for hub in part) - 1):
        for l in part:
            sp_part = substract(part,[l])
            for t in substract(Hubs,part):
                problem += lpSum(Z_var[key0][key1] for key0,key1 in dot(part,part)) <= lpSum(L_var[hub] for hub in sp_part) + 1 - L_var[t]
            

#Constraint 6
for office in End_offices:
    problem += lpSum(X_var[customer][office] for customer in Customers) <= u_index[office]

#Constraint 7
for hub in Hubs:
    problem += lpSum(W_var[key[0]][key[1]][hub] for key in h_index) <= v_index[hub]
    for customer in Customers:
        for office in End_offices:
            problem += W_var[customer][office][hub] <= X_var[customer][office]
            problem += W_var[customer][office][hub] <= Y_var[office][hub]
            problem += W_var[customer][office][hub] >= X_var[customer][office] + Y_var[office][hub] -1
    
#Constraint 8
problem += lpSum(L_var[hub] for hub in Hubs) >= 3

#Constraint 9
problem += lpSum(X_var[key[0]][key[1]] for key in h_index)/number_of_customers >= alpha

In [50]:
#solve the problem
problem.solve()

-1

In [51]:
#print status
print("Status : ", LpStatus[problem.status])

#print var
#for value in problem.variables():
#    print(value.name, " = ",value.varValue)
    
#print Objective value
print( "Objective value = ",problem.objective.value())

Status :  Infeasible
Objective value =  6531871452.0
